In [5]:
import numpy  as np
from scipy import linalg
import math
from numpy.linalg import inv
#downhill
def downhill(F,xStart,side=0.1,tol=1.0e-6):
    n = len(xStart)                 # Number of variables
    x = np.zeros((n+1,n)) 
    f = np.zeros(n+1)
    
  # Generate starting simplex
    x[0] = xStart
    for i in range(1,n+1):
        x[i] = xStart
        x[i,i-1] = xStart[i-1] + side        
  # Compute values of F at the vertices of the simplex     
    for i in range(n+1): f[i] = F(x[i])
    
  # Main loop
    for k in range(500):
      # Find highest and lowest vertices
        iLo = np.argmin(f)
        iHi = np.argmax(f)       
      # Compute the move vector d
        d = (-(n+1)*x[iHi] + np.sum(x,axis=0))/n
      # Check for convergence
        if math.sqrt(np.dot(d,d)/n) < tol: return x[iLo]
        
      # Try reflection
        xNew = x[iHi] + 2.0*d              
        fNew = F(xNew)        
        if fNew <= f[iLo]:        # Accept reflection 
            x[iHi] = xNew
            f[iHi] = fNew
          # Try expanding the reflection
            xNew = x[iHi] + d               
            fNew = F(xNew)
            if fNew <= f[iLo]:    # Accept expansion
                x[iHi] = xNew
                f[iHi] = fNew
        else:
          # Try reflection again
            if fNew <= f[iHi]:    # Accept reflection
                x[iHi] = xNew
                f[iHi] = fNew
            else:
              # Try contraction
                xNew = x[iHi] + 0.5*d
                fNew = F(xNew)
                if fNew <= f[iHi]: # Accept contraction
                    x[iHi] = xNew
                    f[iHi] = fNew
                else:
                  # Use shrinkage
                    for i in range(len(x)):
                        if i != iLo:
                            x[i] = (x[i] - x[iLo])*0.5
                            f[i] = F(x[i])
    print("Too many iterations in downhill")
    return x[iLo]


    
def f(x):
    #特徵值
    global eVal
    #拉格朗日算符
    lamda =1*10**6
    min_eVal = 0.4
    
    A = np.array([[4*(x[0]**4+x[1]**4), 2*x[1]**4],\
                [2*x[1]**4, 4*x[1]**4]])
    B = np.array([[4*(x[0]**2+x[1]**2), -3*x[1]**2],\
                [-3*x[1]**2, 4*x[1]**2]])
    #解廣義特徵向量及特徵值 A=Lamda*B
    #這裡使用scipy的eig與numpy的eig不同在於numpy的不能解廣義的特徵值
    #及特徵向量
    aeVal, l, r = linalg.eig(A, B, right = True, left =True)
    #找較小的
    aeVal = min(aeVal)
    #取實部scipy的eig算出來會顯示虛部
    eVal = aeVal.real
    #merit function:F(x) + lamda*P(x)
    return x[0]**2 + x[1]**2 + lamda*((max(0, min_eVal - eVal))**2)
    
    






In [6]:
dStart = np.array([1.0, 1.0])
d = downhill(f, dStart, 0.1)
print("d = ", d)
print("eigenvalue = ", eVal)

d =  [1.07512696 0.79924677]
eigenvalue =  0.39999775723762787
